In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 300)
pd.set_option("max_colwidth", 200)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn import preprocessing

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('../Data/TrainYear.csv')
test = pd.read_csv('../Data/TestYear.csv')

In [3]:
print(train.shape, test.shape)

(1970, 300) (610, 300)


In [56]:
def Performance_Stable_Split(train, test):
    
    X_train = train.drop(['SalePrice'], axis=1).values
    Y_train = train.SalePrice.values
    
    X_test = test.drop(['SalePrice'], axis=1).values
    Y_test = test.SalePrice.values

    #Y = np.log(data.SalePrice.values)
    
    MLR = LinearRegression()
    MLR.fit(X_train,Y_train)
    pred = MLR.predict(X_test)
    R2_MLR = r2_score(Y_test, pred)
    
    r = Ridge()
    r.fit(X_train,Y_train)
    Ridge_Score = r.score(X_test,Y_test)
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    alphas = 10**np.linspace(10,-2,100)*0.5
    
    ridgecv= RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
    ridgecv.fit(X_train_scaled, Y_train)
    ridge_best_alpha = Ridge(alpha=round(ridgecv.alpha_,2))
    
    ridge_best_alpha.fit(X_train_scaled, Y_train)
    Ridge_A_Score = ridge_best_alpha.score(X_test_scaled, Y_test)
    
    l = Lasso()
    l.fit(X_train,Y_train)
    Lasso_Score = l.score(X_test,Y_test)
    
    lassocv = LassoCV(alphas=alphas, cv=2, max_iter = 100000)
    lassocv.fit(X_train_scaled, Y_train)
    lasso_best_alpha = Lasso(alpha=round(lassocv.alpha_,2))
    lasso_best_alpha.fit(X_train_scaled, Y_train)
    Lasso_A_Score = lasso_best_alpha.score(X_test_scaled, Y_test)
    
    elastic = ElasticNet(alpha=1, l1_ratio=0.5)
    elastic.fit(X_train_scaled,Y_train)
    Elastic_Score = elastic.score(X_test_scaled, Y_test)
    
    RF_No_Bootstrap = RandomForestRegressor(n_estimators=100, criterion='squared_error', 
                                            bootstrap=False)
    RF_No_Bootstrap.fit(X_train_scaled, Y_train)
    RF_No_Bootstrap_Score = RF_No_Bootstrap.score(X_test_scaled, Y_test)
    
    RF_W_Bootstrap = RandomForestRegressor(n_estimators=100, criterion='squared_error', 
                                            bootstrap=True)
    RF_W_Bootstrap.fit(X_train_scaled, Y_train)
    RF_W_Bootstrap_Score = RF_W_Bootstrap.score(X_test_scaled, Y_test)
    
    gbm = GradientBoostingRegressor()
    gbm.fit(X_train_scaled, Y_train)
    GBM_Score = gbm.score(X_test_scaled, Y_test)
    
    xgb = XGBRegressor()
    xgb.fit(X_train_scaled, Y_train)
    XGB_Score = xgb.score(X_test_scaled, Y_test)
    
    #elas_grid = dict()
    #elas_grid['alpha'] = arange(0.1,1,0.1)
    #elas_grid['l1ratio'] = arange(0.1,1,0.1)
    
    #elasticgrid = GridSearchCV(ElasticNet(), elas_grid, 
    #                           scoring='neg_mean_absolute_error', n_jobs=-1)
    
    #elasticgrid.fit(X_scaled, Y)
    
    #results = search.fit(X_scaled, Y)
    
    #elasticgrid = ElasticNet(alpha=results.best_params['alpha'],
    #                        l1_ratio=results.best_params_['l1_ratio'])
    #elasticgrid.fit(X_scaled, Y)
    #Elastic_Score = elasticgrid.score(X_scaled, Y)
    
       
    d = {'Model': ["Multiple Linear Regression", r, ridge_best_alpha, 
                   l, lasso_best_alpha, elastic, "RF No Bootstrap", 
                   "RF With Bootstrap", "Gradient Boosting",
                  "Xtreme Gradient Boosting"],
         'Performance Yr Split (R2)': [R2_MLR, Ridge_Score, Ridge_A_Score, 
                                Lasso_Score, Lasso_A_Score, Elastic_Score,
                                      RF_No_Bootstrap_Score, RF_W_Bootstrap_Score,
                                      GBM_Score, XGB_Score]}
    
    
    
 #   d = {'Model': [MLR, r, ridge_best_alpha, 
 #                  l, lasso_best_alpha, elastic, "RF No Bootstrap", "RF With Bootstrap",
 #                  "Gradient Boosting", "Xtreme Gradient Boosting"],
 #        'Performance Yr Split (R2)': [R2_MLR, Ridge_Score, Ridge_A_Score, 
 #                               Lasso_Score, Lasso_A_Score, Elastic_Score,
 #                               RF_No_Bootstrap_Score, RF_W_Bootstrap_Score,
 #                               GBM_Score, XGB_Score]}
    
    results = pd.DataFrame(data=d)
    return results



In [57]:
Performance_Stable_Split(train, test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.21953e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.188e+11, tolerance: 1.120e+09
  model = cd_fast.enet_coordinate_descent(


,Model,Performance Yr Split (R2)
0,Multiple Linear Regression,0.904039
1,Ridge(),0.916614
2,Ridge(alpha=201.85),0.900729
3,Lasso(),0.905202
4,Lasso(alpha=352.74),0.906303
5,ElasticNet(alpha=1),0.892630
6,RF No Bootstrap,0.831890
7,RF With Bootstrap,0.899860
8,Gradient Boosting,0.928482
9,Xtreme Gradient Boosting,0.903076


In [49]:
trainyrord = pd.read_csv('../Data/TrainYearOrdinal.csv')
testyrord = pd.read_csv('../Data/TestYearOrdinal.csv')

In [50]:
Performance_Stable_Split(trainyrord, testyrord)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.42601e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.191e+11, tolerance: 1.120e+09
  model = cd_fast.enet_coordinate_descent(


,Model,Performance Yr Split (R2)
0,Multiple Linear Regression,0.896042
1,Ridge(),0.896034
2,Ridge(alpha=152.69),0.896212
3,Lasso(),0.896056
4,Lasso(alpha=266.83),0.898044
5,ElasticNet(alpha=1),0.887990
6,RF No Bootstrap,0.829471
7,RF With Bootstrap,0.898728
8,Gradient Boosting,0.922842
9,Xtreme Gradient Boosting,0.907710


In [ ]:
##Elastic Net is not optimized. SVR needs to be better assessed. 